Web Scrape Example --- Ebay
Xianglu He

In [1]:
#Import Packages
import requests
import urllib
import json
import mysql.connector
import re
from bs4 import BeautifulSoup
import urllib.request
import time
import os
import glob
import pandas as pd

# Part One

In [2]:
#Part I

#Type in the URL Batch Request
url = 'http://numbersapi.com/0..99'
#Get the trivia results through website batch API
request = requests.get(url)
#Save the result in json format named as data
data = request.json()

In [3]:
#Create a data_list variable
data_list = []
#Change the json format into list format 
for key in data:
    data_list.append(data[key])

In [4]:
#Create the final result as a list variable
final_result = []
#Append the number with 3 digits format with the result requested from the API
for i in range(len(data_list)) :
  final_result.append("{0:03}".format(i) + ' - ' + data_list[i])

In [5]:
#Print the result line by line
for i in range(len(final_result)):
    print(final_result[i])

000 - 0 is the atomic number of the theoretical element tetraneutron.
001 - 1 is the number of moons orbiting Earth.
002 - 2 is the number of polynucleotide strands in a DNA double helix.
003 - 3 is cans of Spam consumed every second in the United States.
004 - 4 is the number of completed, numbered symphonies by Johannes Brahms.
005 - 5 is the number of Justices on the Supreme Court of the United States necessary to render a majority decision.
006 - 6 is the highest number on one end of a standard domino.
007 - 7 is the number of types of viruses according to the Baltimore classification.
008 - 8 is the number of principles of Yong in Chinese calligraphy.
009 - 9 is the number of circles of Hell in Dante's Divine Comedy.
010 - 10 is the average thickness of the Arctic ice sheet in feet.
011 - 11 is the approximate periodicity of a sunspot cycle in years.
012 - 12 is the number of ounces in a troy pound (used for precious metals).
013 - 13 is the speed of rush hour traffic on average i

# Part Two

Part A

To include Buy-it-now items, we could add an extension condition filter to the current search engine. The condition would be 'LH_BIN'
The GET request for buy-it-now items is https://www.ebay.com/sch/i.html?_nkw=samsung+tv&rt=nc&LH_BIN=

To include 100 items per search result page, we could add an extension condition filter to the current search engine. The condition would be '_ipg=100'
The GET request for 100 items per search result is https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_ipg=100

In [6]:
#Part B
# pretend I am a browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
url = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&_ipg=100&_pgn='

#Create empyty list to save the links
url_spon = []
url_no = []

#Open the page from 1 to 10 on Ebay and save links into above two lists
for pgn in range(1,11):
    new_url = url + str(pgn)
    # open a new session
    session = requests.Session() 
    # log on to the session and get the content of said url 
    response = session.get(new_url,headers=headers) 
    test = response.text
    soup = BeautifulSoup(test, 'html.parser') 
    page_ind = soup.select('li.s-item')
    #Find the class that has unique pattern to identify sponsored links 
    for page_ind in page_ind:
        a = page_ind.find('a')
        if not a:
            continue
        page_ind = page_ind.find('span').get_text()
        #The unique pattern
        if bool(re.search(r'.*[S].*[P].*[O].*[N].*[S].*[O].*[R].*[E].*[D].*',page_ind)) == True:
            #If the unique pattern is true, then save the link into Sponsored list
            url_spon.append(re.sub("(.*)\\?.*", r"\1", a['href']))
        #Else, save it into Non_Sponsored List
        else:
            url_no.append(re.sub("(.*)\\?.*", r"\1", a['href']))
    #10 Second Gap between each request        
    time.sleep(10)
                

#Save the sponsored links into a txt file called sponsored
with open('sponsored.txt', 'w') as filehandle:
    for listitem in url_spon:
        filehandle.write('%s\n' % listitem)
        
#Save the non_sponsored links into a txt file called non_sponsored
with open('non-sponsored.txt', 'w') as filehandle:
    for listitem in url_no:
        filehandle.write('%s\n' % listitem)       


In [7]:
#Part C

#Create a folder function 
def createFolder(directory):
    try:
        #if the folder is not existed, then create the folder
        if not os.path.exists(directory):
            os.makedirs(directory)
    #Error handling
    except OSError:
            print ('Error: Creating directory. ' +  directory)

#Create sponsored and non-sponsored folders
createFolder('./sponsored/')
createFolder('./non-sponsored/')

#Open the text file and strip the url into two lists
sponsored_url = open('sponsored.txt','r')
sponsored = []
for x in sponsored_url:
  sponsored.append(x.strip('\n'))

non_sponsored_url = open('non-sponsored.txt','r')
non_sponsored = []
for x in non_sponsored_url:
  non_sponsored.append(x.strip('\n'))

In [8]:
#Save the sponsored html file
for pgn in range(len(sponsored)):
    try:
        #Default path
        save_path = './sponsored/'
        #Get the current new url
        url = sponsored[pgn]
        #Strip the item id from url
        filename = re.findall(r'/([0-9]+)\Z',url)
        filename = filename[0]
        #Save the name as default path + item id 
        completeName = os.path.join(save_path,filename+".html")
        #Get the url request
        response = requests.get(url, headers = headers)
        #Write the request and save it into the default path
        with open(completeName, 'w',encoding='utf8') as file:
             file.write(response.text)
    except ConnectionError as e: 
        print(e)
    #2 second time gap
    time.sleep(2)

In [9]:
#Save the non-sponsored html file
for pgn in range(len(non_sponsored)):
    try:
        #Default path
        save_path = './non-sponsored/'
        #Get the current new url
        url = non_sponsored[pgn]
        #Strip the item id from url
        filename = re.findall(r'/([0-9]+)\Z',url)
        filename = filename[0]
        #Save the name as default path + item id
        completeName = os.path.join(save_path,filename+".html")
        #Get the url request
        response = requests.get(url, headers = headers)
        #Write the request and save it into the default path
        with open(completeName, 'w',encoding='utf8') as file:
             file.write(response.text)
    except ConnectionError as e: 
        print(e)
    #2 second time gap
    time.sleep(2)

In [10]:
#Sponsored Section
#Create empty lists to store the value
seller_name = []
seller_score = []
item_price = []
item_sold = []
best_offer = []
title = []
returns = []
shipping = []
condition = []
Type = []
#Default path and look for all of the html file
list_of_files = glob.glob('./sponsored/*.html')
for file_name in list_of_files:
    #Open the html file
    file = open(file_name, 'r',encoding='utf8') 
    text = file.read()
    #Parse the information from the current url
    soup = BeautifulSoup(text,'html.parser')
    
    #If soup has the seller_name information, append the value into seller_name list
    #Seller_name is saved under span class called mbg-nw
    if bool(soup.find('span',{'class':'mbg-nw'})) == True: 
        seller_name.append(soup.find('span',{'class':'mbg-nw'}).get_text())
    #If soup does not have the information, save it as NULL
    else: 
        seller_name.append('NULL')
        
    #If soup has the seller_score information, append the value into seller_score list
    #Seller_score is saved under span class mbg-l
    if bool(soup.find('span',{'class':'mbg-l'})) == True:
        seller_score.append(re.findall('[0-9]+',soup.find('span',{'class':'mbg-l'}).get_text())[0])
    #If soup does not have the information, save it as NULL
    else: 
        seller_score.append('0')
        
    #If soup has the item_price information, append the value into item_price list
    #Item_price is saved under span class notranslate 
    if bool(soup.find('span',{'class':'notranslate'})) == True:
        #If the item_price has only one value
        if len(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text())) ==1:
            #item_price.append(int(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text())[0])*100))
            #Save the value into item_price list
            item_price.append(''.join(re.findall('[0-9]',soup.find('span',{'class':'notranslate'}).get_text())))
        #else:item_price.append(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text()))
        #Save the value into item_price list
        else: item_price.append(''.join(re.findall('[0-9]',soup.find('span',{'class':'notranslate'}).get_text())))
    #If soup does not have the information, save it as 000
    else: 
        item_price.append('000')
        
    #If soup has the item_sold information, append the value into item_sold list
    #Item_sold is saved under a class vi-txt-underline
    if bool(soup.find('a',{'class':'vi-txt-underline'})) == True:
        #Save the value into item_sold list
        item_sold.append(re.findall('[0-9]+',soup.find('a',{'class':'vi-txt-underline'}).get_text())[0])
    #If soup does not have the information, save it as 0
    else: 
        item_sold.append('0')
        
    #If soup has the best_offer information, append the value into best_offer list
    #Best_offer is saved under div class vi-bbox-dspn u-flL lable boLable
    if bool(soup.find('div',{'class':'vi-bbox-dspn u-flL lable boLable'})) == True:
        #Save the value as True into best_offer list
        best_offer.append('True')
    
    else: 
        best_offer.append('False')
    
    #If soup has the title information, append the value into title list
    #Title is saved under h1 class it-ttl
    if bool(soup.find('h1',{'class':'it-ttl'})) == True:
        #Add the [0] position to remove the bracket []
        title.append(re.findall(r'</span>(.+)</h1>',str(soup.find('h1',{'class':'it-ttl'})))[0])
    #If soup does not have the information, save it as NULL
    else: title.append('NULL')
        
    #If soup has the returns information, append the value into returns list
    #Returns is saved under span id vi-ret-accrd-txt
    if bool(soup.find('span',{'id':'vi-ret-accrd-txt'})) == True:
        returns.append(soup.find('span',{'id':'vi-ret-accrd-txt'}).get_text())
    #If soup does not have the information, save it as NULL
    else: returns.append('NULL')
        
    #If soup does not find the pattern, save it as 000
    #Shipping is saved under span id fshippingCost
    if soup.find('span',attrs = {'id':'fshippingCost'}) == None:
        shipping.append('000')
    #If soup has the pattern
    elif bool(soup.find('span',{'id':'fshippingCost'})) == True:
        #Shipping price has more than one price, this is sometimes because of different currency
        if len(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text()))>1:
            #Get the second position price, which is the US dollars and save it into shipping
            shipping.append(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text())[1])*100)
        
        else:
            #The pattern has been found but return as empty, this is because it various by locations 
            if re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text()) == []:
                #Save this condition as 000
                shipping.append('000')
            #Get the first position because it only has one shipping price and save it into shipping list
            else: 
                shipping.append(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text())[0])*100)
    #Save the result of conditions into 000 as error handling
    else: 
        shipping.append('000')
    #Get the condition type and save it into condition list
    #Condition is saved under div class u-flL condText
    condition.append(soup.find('div',{'class':'u-flL condText'}).get_text())
    #Save the type as sponsored
    Type = 'sponsored'

In [11]:
#Concate all the columns and save it into a sponsored dataframe
sponsored_df = pd.DataFrame({'seller_name': seller_name,
                             'seller_score': seller_score,
                             'item_price': item_price,
                             'item_sold': item_sold,
                             'best_offer': best_offer,
                             'title': title,
                             'returns': returns,
                             'shipping': shipping,
                             'condition': condition,
                             'type': Type},
                           columns=['seller_name','seller_score','item_price','item_sold','best_offer','title','returns','shipping','condition','type'])

In [12]:
#Non_Sponsored Section
#Create empty lists to store the value
seller_name = []
seller_score = []
item_price = []
item_sold = []
best_offer = []
title = []
returns = []
shipping = []
condition = []
Type = []
#Default path and look for all of the html file
list_of_files = glob.glob('./non-sponsored/*.html')
for file_name in list_of_files:
    #Open the html file
    file = open(file_name, 'r',encoding='utf8') 
    text = file.read()
    #Parse the information from the current url
    soup = BeautifulSoup(text,'html.parser')
    
    #If soup has the seller_name information, append the value into seller_name list
    #Seller_name is saved under span class called mbg-nw
    if bool(soup.find('span',{'class':'mbg-nw'})) == True: 
        seller_name.append(soup.find('span',{'class':'mbg-nw'}).get_text())
    #If soup does not have the information, save it as NULL
    else: 
        seller_name.append('NULL')
        
    #If soup has the seller_score information, append the value into seller_score list
    #Seller_score is saved under span class mbg-l
    if bool(soup.find('span',{'class':'mbg-l'})) == True:
        seller_score.append(re.findall('[0-9]+',soup.find('span',{'class':'mbg-l'}).get_text())[0])
    #If soup does not have the information, save it as 0
    else: 
        seller_score.append('0')
        
    #If soup has the item_price information, append the value into item_price list
    #Item_price is saved under a class notranslate
    if bool(soup.find('span',{'class':'notranslate'})) == True:
        #If the item_price has only one value
        if len(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text())) ==1:
            #item_price.append(int(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text())[0])*100))
            #Save the value into item_price as joining the number one by one 
            item_price.append(''.join(re.findall('[0-9]',soup.find('span',{'class':'notranslate'}).get_text())))
        #else:item_price.append(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',{'class':'notranslate'}).get_text()))
        #If the item_price has more than one value case
        else: item_price.append(''.join(re.findall('[0-9]',soup.find('span',{'class':'notranslate'}).get_text())))
    #Save the value as 000 for all the left cases
    else: 
        item_price.append('000')
        
    #If soup has the item_sold information, append the value into item_sold list
    #Item_sold is saved under a class vi-txt-underline
    if bool(soup.find('a',{'class':'vi-txt-underline'})) == True:
        item_sold.append(re.findall('[0-9]+',soup.find('a',{'class':'vi-txt-underline'}).get_text())[0])
    #If soup does not have the information, save it as 0
    else: 
        item_sold.append('0')
        
    #If soup has the best_offer information, append the value into best_offer list
    #Best_offer is saved under div class vi-bbox-dspn u-flL lable boLable
    if bool(soup.find('div',{'class':'vi-bbox-dspn u-flL lable boLable'})) == True:
        best_offer.append('True')
    #If soup does not have the information, save it as FALSE
    else: 
        best_offer.append('False')
        
    #If soup has the title information, append the value into title list
    #Title is saved under h1 class it-ttl
    if bool(soup.find('h1',{'class':'it-ttl'})) == True:
        title.append(str(soup.find('h1',{'class':'it-ttl'})))
    #If soup does not have the information, save it as NULL
    else: title.append('NULL')
        
    #If soup has the returns information, append the value into returns list
    #Returns is saved under span id vi-ret-accrd-txt    
    if bool(soup.find('span',{'id':'vi-ret-accrd-txt'})) == True:
        returns.append(soup.find('span',{'id':'vi-ret-accrd-txt'}).get_text())
    #If soup does not have the information, save it as NULL
    else: returns.append('NULL')
        
    #If soup does not find the pattern, save it as 000
    #Shipping is saved under span id fshippingCost
    if soup.find('span',attrs = {'id':'fshippingCost'}) == None:
        shipping.append('000')
    #If soup has the pattern
    elif bool(soup.find('span',{'id':'fshippingCost'})) == True:
        #Shipping price has more than one price, this is sometimes because of different currency
        if len(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text()))>1:
            #Get the second position price, which is the US dollars and save it into shipping
            shipping.append(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text())[1])*100)
        else:
            #The pattern has been found but return as empty, this is because it various by locations 
            if re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text()) == []:
                shipping.append('000')
            #Get the first position because it only has one shipping price and save it into shipping list
            else: 
                shipping.append(float(re.findall('\$([0-9]+\.[0-9]+)',soup.find('span',attrs = {'id':'fshippingCost'}).get_text())[0])*100)
    #Save the result of conditions into 000 as error handling
    else: 
        shipping.append('000')
    #Get the condition type and save it into condition list
    #Condition is saved under div class u-flL condText
    if bool(soup.find('div',{'class':'u-flL condText'})) == True:
        condition.append(soup.find('div',{'class':'u-flL condText'}).get_text())
    else: condition.append('NULL')
    #Save the type as sponsored
    Type = 'non-sponsored'

In [13]:
#Remove the bracket inside the non_sponsored title column
for i in range(len(title)):
    title[i] = re.findall(r'</span>(.+)</h1>',title[i])
    title[i] = str(title[i]).replace('[','').replace(']','')

In [14]:
#Concate all the columns and save it into a non_sponsored dataframe
non_sponsored_df = pd.DataFrame({'seller_name': seller_name,
                             'seller_score': seller_score,
                             'item_price': item_price,
                             'item_sold': item_sold,
                             'best_offer': best_offer,
                             'title': title,
                             'returns': returns,
                             'shipping': shipping,
                             'condition': condition,
                             'type': Type},
                           columns=['seller_name','seller_score','item_price','item_sold','best_offer','title','returns','shipping','condition','type'])

In [15]:
#Concate sponsored_df and non_sponsored_df as a single final dataframe
frame = [sponsored_df,non_sponsored_df]
final_dataframe = pd.concat(frame)
#Reformate the return policy into Yes or No
for i in range(len(final_dataframe)):
    if final_dataframe.iloc[i,6] == 'Seller does not accept returns':
        final_dataframe.iloc[i,6] = 'NO'
    else:  final_dataframe.iloc[i,6] = 'YES'
final_dataframe.head()

,seller_name,seller_score,item_price,item_sold,best_offer,title,returns,shipping,condition,type
0,vipoutlet,258955,26900,58,False,Sony CUH-2215A PlayStation 500GB Slim System P...,YES,000,Used,sponsored
1,ezprobay99,4009,25499,36,False,Brand New Sony PlayStation 4 Slim 1TB -- Only...,YES,000,New,sponsored
2,normabreede0,15,45000,0,False,Sony PlayStation 4 PS4 Slim 1TB Console Marvel...,NO,000,New,sponsored
3,monstertechbids,15342,23999,21,False,Sony Playstation 4 PS4 Slim 1TB Jet Black Cons...,YES,000,Used,sponsored
4,alejandrosolanocleats,17,15300,0,True,Playstation 4 500GB Slim - Black - Great condi...,NO,3500,Used,sponsored


In [23]:
#Connect it to the local SQL Server
mydb = mysql.connector.connect(
  host="localhost",
  user="root"
)

mycursor = mydb.cursor()

#Create Database, table and variables
mycursor.execute("CREATE DATABASE IF NOT EXISTS eBay")
mycursor.execute("USE eBay")
mycursor.execute("CREATE TABLE IF NOT EXISTS eBay_items (id INT AUTO_INCREMENT PRIMARY KEY,seller_name VARCHAR(255),seller_score int, item_price int, item_sold int, best_offer VARCHAR(25), title VARCHAR(9999), returns VARCHAR(255), shipping int, Type_Condition VARCHAR(25), Item_Type VARCHAR(25))")



#Loop the dataframe and add value into each of the variable
for i in range(len(final_dataframe)):
    sql = 'INSERT INTO eBay_items (seller_name, seller_score, item_price, item_sold, best_offer, title, returns, shipping,Type_Condition,Item_Type) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    val = final_dataframe.iloc[i,0],final_dataframe.iloc[i,1],final_dataframe.iloc[i,2],final_dataframe.iloc[i,3],final_dataframe.iloc[i,4],final_dataframe.iloc[i,5],final_dataframe.iloc[i,6],final_dataframe.iloc[i,7],final_dataframe.iloc[i,8],final_dataframe.iloc[i,9]
    mycursor.execute(sql,val)

mydb.commit()  
    
    
mydb.close()

In [24]:
#Part F
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database = "ebay"
)
#Get the stats of seller_score
mycursor = mydb.cursor()
mycursor.execute("SELECT Item_Type, Type_Condition, avg(seller_score),max(seller_score), min(seller_score) FROM eBay_items GROUP BY Item_Type,Type_Condition")
data = mycursor.fetchall()
print("facts for seller_score: ")
print('Item_Type  -  Condition  -  Mean  -        Max  -   Min')
for i in data:
    print(i)

facts for seller_score: 
Item_Type  -  Condition  -  Mean  -        Max  -   Min
('sponsored', 'Used', Decimal('21560.3500'), 258955, 0)
('sponsored', 'New', Decimal('15844.0222'), 568396, 0)
('sponsored', 'Seller refurbished', Decimal('15099.2500'), 41457, 0)
('sponsored', 'Open box', Decimal('11246.5000'), 64483, 5)
('sponsored', 'Manufacturer refurbished', Decimal('56.0000'), 56, 56)
('non-sponsored', 'Seller refurbished', Decimal('160004.3235'), 2265933, 0)
('non-sponsored', 'Used', Decimal('41001.5717'), 258958, 0)
('non-sponsored', 'New', Decimal('14165.8805'), 916886, 0)
('non-sponsored', 'Open box', Decimal('14498.5893'), 568398, 0)
('non-sponsored', 'Manufacturer refurbished', Decimal('283314.3750'), 2265917, 0)


In [25]:
#Get the stats of item_price
mycursor = mydb.cursor()
mycursor.execute("SELECT Item_Type, Type_Condition, avg(item_price),max(item_price), min(item_price) FROM eBay_items GROUP BY Item_Type,Type_Condition")
data = mycursor.fetchall()
print("facts for item_price: ")
print('Item_Type  -  Condition  -  Mean  -        Max  -   Min')
for i in data:
    print(i)

facts for item_price: 
Item_Type  -  Condition  -  Mean  -        Max  -   Min
('sponsored', 'Used', Decimal('27180.1000'), 55351, 14599)
('sponsored', 'New', Decimal('32275.7333'), 57999, 22100)
('sponsored', 'Seller refurbished', Decimal('22999.5000'), 25300, 19999)
('sponsored', 'Open box', Decimal('24215.0000'), 33500, 16995)
('sponsored', 'Manufacturer refurbished', Decimal('33900.0000'), 33900, 33900)
('non-sponsored', 'Seller refurbished', Decimal('21571.4412'), 36995, 7100)
('non-sponsored', 'Used', Decimal('20615.6667'), 50000, 99)
('non-sponsored', 'New', Decimal('31152.1352'), 157500, 4600)
('non-sponsored', 'Open box', Decimal('24741.9821'), 52995, 12500)
('non-sponsored', 'Manufacturer refurbished', Decimal('18506.0000'), 25000, 10250)


In [26]:
#Get the stats of item_sold
mycursor = mydb.cursor()
mycursor.execute("SELECT Item_Type, Type_Condition, avg(item_sold),max(item_sold), min(item_sold) FROM eBay_items GROUP BY Item_Type,Type_Condition")
data = mycursor.fetchall()
print("facts for item_sold: ")
print('Item_Type  -  Condition  -  Mean  -        Max  -   Min')
for i in data:
    print(i)

facts for item_sold: 
Item_Type  -  Condition  -  Mean  -        Max  -   Min
('sponsored', 'Used', Decimal('1.5000'), 58, 0)
('sponsored', 'New', Decimal('7.5778'), 99, 0)
('sponsored', 'Seller refurbished', Decimal('1.2500'), 4, 0)
('sponsored', 'Open box', Decimal('0.8333'), 5, 0)
('sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0)
('non-sponsored', 'Seller refurbished', Decimal('2.7059'), 43, 0)
('non-sponsored', 'Used', Decimal('0.2597'), 58, 0)
('non-sponsored', 'New', Decimal('9.7516'), 872, 0)
('non-sponsored', 'Open box', Decimal('2.2857'), 119, 0)
('non-sponsored', 'Manufacturer refurbished', Decimal('1.0000'), 5, 0)


In [27]:
#Get the stats of best_offer
mycursor = mydb.cursor()
mycursor.execute("SELECT Item_Type, Type_Condition, count(best_offer) FROM eBay_items WHERE best_offer = 'FALSE' GROUP BY Item_Type,Type_Condition")
data = mycursor.fetchall()
print("facts for best_offer is FALSE: ")
print('Item_Type  -  Condition  -  Count')
for i in data:
    print(i)

facts for best_offer is FALSE: 
Item_Type  -  Condition  -  Count
('sponsored', 'Used', 28)
('sponsored', 'New', 34)
('sponsored', 'Seller refurbished', 3)
('sponsored', 'Open box', 1)
('non-sponsored', 'Seller refurbished', 20)
('non-sponsored', 'Used', 304)
('non-sponsored', 'New', 231)
('non-sponsored', 'Open box', 26)
('non-sponsored', 'Manufacturer refurbished', 3)


In [28]:
#Get the stats of returns
mycursor = mydb.cursor()
mycursor.execute("SELECT Item_Type, Type_Condition, count(returns) FROM eBay_items WHERE returns = 'YES' GROUP BY Item_Type,Type_Condition")
data = mycursor.fetchall()
print("facts for returns is YES: ")
print('Item_Type  -  Condition  -  Count')
for i in data:
    print(i)

facts for returns is YES: 
Item_Type  -  Condition  -  Count
('sponsored', 'Used', 33)
('sponsored', 'New', 33)
('sponsored', 'Seller refurbished', 3)
('sponsored', 'Open box', 3)
('non-sponsored', 'Seller refurbished', 17)
('non-sponsored', 'Used', 229)
('non-sponsored', 'New', 163)
('non-sponsored', 'Manufacturer refurbished', 3)
('non-sponsored', 'Open box', 14)


In [29]:
#Close the connection with SQL Server
mydb.close()

#G
By looking at the stats fact that generated in part F, there are two ways to classify sponsored and non-sponsored types. First way is looking at the seller_score. The other difference is item_price. In seller_score section, under each condition, the seller who sells non-sponsored type would generally has more scores than the the one who sells sponsored type. On the other hand, the condition surprsingly changes in item_price. Sponsored type generally has higher price than the non-sponsored type, with roughly 60-100$ excess.